<a href="https://colab.research.google.com/github/ArshT/Pytorch_Practice/blob/master/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision.transforms import transforms

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5),(0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,batch_size = 32,shuffle=True,num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset,batch_size = 32,shuffle=True,num_workers=2)

In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

device = torch.device("cuda:0" if USE_GPU else "cpu")

using device: cuda


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(1,32,(3,3),(1,1),padding=(1,1))
    self.batch1 = nn.BatchNorm2d(32)

    self.conv2 = nn.Conv2d(32,64,(3,3),(1,1),padding=(1,1))
    self.batch2 = nn.BatchNorm2d(64)

    self.pool = nn.MaxPool2d(2,2)

    self.drop = nn.Dropout(0.2)
    self.fc1 = nn.Linear(64*7*7,512)
    self.fc2 = nn.Linear(512,256)
    self.fc3 = nn.Linear(256,128)
    self.fc4 = nn.Linear(128,10)
  
  def forward(self,x):

    x = self.batch1(self.conv1(x))
    x = F.relu(x)
    x = self.pool(x)

    x = self.batch2(self.conv2(x))
    x = F.relu(x)
    x = self.pool(x)

    x = x.view(x.shape[0],64*7*7)

    x = F.relu(self.fc1(x))
    x = self.drop(x)
    x = F.relu(self.fc2(x))
    x = self.drop(x)
    x = F.relu(self.fc3(x))
    x = self.drop(x)
    x = self.fc4(x)

    return x
  
net = Net()
net = net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.95)

In [ ]:
def model_eval(dataloader):
    total = 0
    correct = 0
    for data in dataloader:
      images, l = data
      images = images.to(device)
      l = l.to(device)

      out = net(images)
      max_val, preds = torch.max(out,dim=1)
      
      total += l.shape[0]                   
      correct += (preds == l).sum().item()  
      accuracy = (100 * correct)/total
    
    
    return accuracy 

In [ ]:
for epoch in range(10):

  running_loss = 0.0
  for i,data in enumerate(trainloader,0):

    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    
    #train_acc = model_eval(trainloader)
    #test_acc = model_eval(testloader)
    
    if i % 300 == 299:    
      print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 300))
      #print(train_acc,test_acc)
      running_loss = 0.0
    
  train_acc = model_eval(trainloader)
  test_acc = model_eval(testloader)

  print("Training Accuracy:",train_acc,"Testing Accuracy",test_acc)
  print("")

[1,   300] loss: 0.035
[1,   600] loss: 0.032
[1,   900] loss: 0.042
[1,  1200] loss: 0.043
[1,  1500] loss: 0.034
[1,  1800] loss: 0.040
Training Accuracy: 99.295 Testing Accuracy 99.13

[2,   300] loss: 0.023
[2,   600] loss: 0.029
[2,   900] loss: 0.034
[2,  1200] loss: 0.030
[2,  1500] loss: 0.030
[2,  1800] loss: 0.027
Training Accuracy: 99.45166666666667 Testing Accuracy 99.02

[3,   300] loss: 0.017
[3,   600] loss: 0.022
[3,   900] loss: 0.029
[3,  1200] loss: 0.021
[3,  1500] loss: 0.031
[3,  1800] loss: 0.025
Training Accuracy: 99.475 Testing Accuracy 99.26

[4,   300] loss: 0.018
[4,   600] loss: 0.018
[4,   900] loss: 0.019
[4,  1200] loss: 0.019
[4,  1500] loss: 0.021
[4,  1800] loss: 0.025
Training Accuracy: 99.54 Testing Accuracy 99.31

[5,   300] loss: 0.013
[5,   600] loss: 0.014
[5,   900] loss: 0.022
[5,  1200] loss: 0.024
[5,  1500] loss: 0.018
[5,  1800] loss: 0.016
Training Accuracy: 99.63166666666666 Testing Accuracy 99.14

[6,   300] loss: 0.016
[6,   600] loss:

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
PATH = '/content/gdrive/My Drive/Pytorch_Practice/MNIST_Model-1.pt'
torch.save(net,PATH)